In [1]:
import numpy as np
import cv2

In [2]:
face_detection_model=cv2.dnn.readNetFromCaffe("./models/deploy.prototxt.txt","./models/res10_300x300_ssd_iter_140000_fp16.caffemodel")

In [3]:
def face_blurring(original_img):
    image=original_image.copy()

    #face_mask is essentailly a black image with white spapces where the faces are  detected
    face_mask=np.zeros(image.shape[:2],dtype='uint8')
    # getting blob from image
    blob=cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
    face_detection_model.setInput(blob)
    detections=face_detection_model.forward()
    h,w=image.shape[:2]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            box=box.astype(int)
            point1=(box[0],box[1])
            point2=(box[2],box[3])
            cv2.rectangle(face_mask,point1,point2,255,-1)
    
    inverse_facemask=cv2.bitwise_not(face_mask)
    bg_image=cv2.bitwise_and(original_image,original_image,mask=face_mask)
    fg_image=cv2.bitwise_and(original_image,original_image,mask=inverse_facemask)
    bg_blur=cv2.GaussianBlur(bg_image,(101,101),0)
    face_blur=cv2.add(bg_blur,fg_image)
    
    return face_blur

In [7]:
cap=cv2.VideoCapture(0)

while True:
    ret,original_image=cap.read()
    
    if ret==False:
        break
    feed=face_blurring(frame)
    cv2.imshow('live blurred feed',feed)
    if cv2.waitKey(1)==27:
        break
    
cap.release
cv2.destroyAllWindows()